In [77]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from google.cloud import storage

import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
spark = SparkSession.builder.appName('ParquetExample').getOrCreate()
spark

In [79]:
client = storage.Client()
# https://cloud.google.com/storage/docs/samples/storage-list-files#storage_list_files-python

In [82]:
parquet_bucket_path = "parkinsons_data/unlabeled"
parquet_bucket_name = "msca-bdp-student-gcs"
full_path = "msca-bdp-student-gcs/parkinsons_data/unlabeled"

In [83]:
blobs = client.list_blobs(parquet_bucket_name, prefix=parquet_bucket_path)

In [84]:
blobs

In [85]:
from pyspark.sql.functions import lit

combined_df = None
for blob in blobs:
    file_name = blob.name.split("/")[-1].split(".")[0]
    print("file name:", file_name)
    df = spark.read.parquet(f"gs://{parquet_bucket_name}/{blob.name}")
    # add file name as column - https://sparkbyexamples.com/pyspark/pyspark-lit-add-literal-constant/
    df = df.withColumn("file_name", lit(file_name))
    if combined_df is None:
        combined_df = df
    else:
#         https://www.geeksforgeeks.org/how-to-union-multiple-dataframe-in-pyspark/
        combined_df = combined_df.union(df)

   

file name: 00c4c9313d
file name: 07a96f89ec
file name: 0d1bc672a8
file name: 0e333c9833
file name: 164adaed7b
file name: 17e0c0dc86
file name: 1c3719ea59
file name: 1cf80df2d6
file name: 24016102f2
file name: 276630050d
file name: 28e6c306ba
file name: 2caa348298
file name: 32bdbc35a0
file name: 3ae6b0f79f
file name: 3bd159ded0
file name: 3f51a63612
file name: 3fc03f01ed
file name: 3fe2624b51
file name: 40bf6c162f
file name: 418a1ca2c1
file name: 43ac46d679
file name: 48081794eb
file name: 48b636e0f5
file name: 4b84027351
file name: 4e44a97a85
file name: 52fd07ea27
file name: 5535c94fc9
file name: 57741bad42
file name: 5bf570bb7b
file name: 5e13d48878
file name: 6e0303484e
file name: 6ed2f109c3
file name: 74f1e1e0ba
file name: 7ab610bb34
file name: 831c13620e
file name: 88f67f91db
file name: 8959244e1c
file name: 8ca674a988
file name: 924e997065
file name: 93abd37fee
file name: 96f57b4a40
file name: 9da3e3dc66
file name: 9fb7805d99
file name: a213c90b02
file name: b15168b788
file name:

In [73]:
combined_df.printSchema()


root
 |-- Time: long (nullable = true)
 |-- AccV: double (nullable = true)
 |-- AccML: double (nullable = true)
 |-- AccAP: double (nullable = true)
 |-- file_name: string (nullable = false)



In [72]:
num_partitions = combined_df.rdd.getNumPartitions()
num_partitions

550

In [75]:
from pyspark.sql.functions import max

max_time = df.select(max("Time")).collect()[0][0]
max_time

## need to do more robustness checking on some sample files 
# to make sure the join actually worked. Maybe run this by file_id,
# there's a decent amount of variation in amount of data provided by person
# (some files are way bigger than others by person)


60479669

In [76]:
max_AccV = df.select(max("AccV")).collect()[0][0]
max_AccV

4.20992160917169

In [87]:
max_pesron = df.select(max("file_name")).collect()[0][0]
max_pesron

'f3fa3bf72d'